In [0]:
dbutils.widgets.text('ServerlessJobID', 'MISSING', 'Serverless Job ID')
dbutils.widgets.text('ClassicJobID', 'MISSING', 'Classic Job ID')

ServerlessJobID = dbutils.widgets.get('ServerlessJobID')
ClassicJobID = dbutils.widgets.get('ClassicJobID')

assert ServerlessJobID != 'MISSING', "Need to provide Job ID for Serverless Job"
assert ClassicJobID != 'MISSING', "Need to provide Job ID for Classic Job"

In [0]:
%sql
with latest_price as (
  select
    sku_name,
    pricing.default as list_price,
    row_number() over (partition by sku_name order by price_start_time desc) as rn
  FROM
    system.billing.list_prices
),
jobs_metadata AS (
  select
    job_id,
    `name` as job_name,
    row_number() over (partition by job_id order by change_time desc) as rn
  from
    system.lakeflow.jobs
  WHERE
    `job_id` IN (:ServerlessJobID, :ClassicJobID)
),
job_run_metadata AS (
  select
    jrt.run_id,
    jrt.period_start_time as period_start_time,
    jrt.job_parameters,
    row_number() over (partition by jrt.run_id order by period_start_time) as rn
  from
    system.lakeflow.job_run_timeline jrt
  WHERE jrt.job_id IN (:ServerlessJobID, :ClassicJobID)
),
latest_job_status AS (
  select
    jrt.run_id,
    jrt.period_end_time as period_end_time,
    jrt.termination_code,
    row_number() over (partition by jrt.run_id order by period_start_time DESC) as rn
  from
    system.lakeflow.job_run_timeline jrt
  WHERE jrt.job_id IN (:ServerlessJobID, :ClassicJobID)
)
select
  job_name,
  sum(u.usage_quantity) as dbus_emitted,
  u.usage_metadata.job_id,
  u.usage_metadata.job_run_id,
  CONVERT_TIMEZONE('UTC', 'Asia/Singapore', jrm.period_start_time) as job_run_start,
  CONVERT_TIMEZONE('UTC', 'Asia/Singapore', latest_job_status.period_end_time) as job_run_end,
  jrm.job_parameters,
  latest_job_status.termination_code,
  lp.list_price,
  sum(u.usage_quantity * lp.list_price) as list_cost
from
  system.billing.usage u
    left join latest_price lp
      on u.sku_name = lp.sku_name
      and lp.rn = 1
    inner join jobs_metadata job
      on u.usage_metadata.job_id = job.job_id
      and job.rn = 1
    inner join job_run_metadata jrm
      on u.usage_metadata.job_run_id = jrm.run_id
      and jrm.rn = 1
    inner join latest_job_status
      ON latest_job_status.run_id = jrm.run_id
      and latest_job_status.rn = 1
WHERE
  `usage_date` >= '2025-08-18' 
GROUP BY
  ALL
ORDER BY
  job_run_start;